In [1]:
# Import des packages

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

import time
import pandas as pd
import os

# from selenium.webdriver.support.ui import Select


In [2]:
# URL de la page souhaitée

url = "https://www.sustainalytics.com/esg-ratings"

# Où seront stockée les données
output_file = "real_scraped_data.csv"


In [ ]:
# Fonction qui sauvegarde les données au fur et à mesure du scraping dans un DF

def save_data(data, file_path):
    df = pd.DataFrame(data)
    if os.path.exists(file_path):
        df.to_csv(file_path, index=False, mode='a', header=False)
    else:
        df.to_csv(file_path, index=False)

# Fonction qui scrape les détails de chaque entreprise

def scrape_company_details(driver, wait):
    try:
        # La suite du programme se lance une fois qu'elle est bien chargée
        grade = wait.until(EC.presence_of_element_located(
            (By.CLASS_NAME, "risk-rating-score"))).text
        
        # On récupère le pays de la firme
        country = driver.find_element(By.CLASS_NAME, "country").text

        # On récupère la date de la dernière mise à jour 
            # Ici on mets la condition "try" car il y a des entreprises qui n'ont pas de date de mise à jour
        try:
            date = driver.find_element(By.XPATH, "//span[contains(text(), 'Last Full Update')]/strong").text
        except Exception:
            date = "Non disponible"

        # On récupère le secteur de la firme
        try:
            industry = driver.find_element(By.CSS_SELECTOR, "strong.industry-group").text
        except Exception:
            industry = "Non disponible"

        # On récupère le nombre d'employé de la firme
        try:
            employees = driver.find_element(By.XPATH, "//div[contains(text(), 'Full time employees')]/strong").text
        except Exception:
            employees = "Non disponible"

        return {"grade": grade, "country": country, "date": date, "industry": industry, "employees": employees}
    except (TimeoutException, StaleElementReferenceException) as e:
        print(f"Erreur détails: {e}")
        return None

# Fonction qui scrape les entreprises
def scrape_companies(url, num_pages): # On y indique l'url et le nombre de pages que l'on souhaite scraper
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)
    data = [] # On initialise une liste vide pour stocker les données
    
    try:
        driver.get(url)
        
        for page in range(num_pages):
            print(f"Scraping page {page + 1}")
            
            # Attendre que les éléments soient chargés
            company_links = wait.until(EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "a.primary-color.d-block.js-fix-path")))

            companies = []
            for link in company_links:
                try:
                    name = link.text
                    href = link.get_attribute('href') # On récupère le lien sur lequel il faudra ensuite cliquer
                    # pour accéder aux détails de l'entreprise
                    companies.append((name, href)) # On ajoute tout les noms de firme de la même page dans une liste
                except StaleElementReferenceException:
                    print("Lien obsolète, ignoré.")

            
            for name, company_url in companies:
                try:
                    # Pour chaqu firme on ouvre un nouvel onglet
                    driver.execute_script("window.open(arguments[0]);", company_url)
                    # Passer à la nouvelle fenêtre
                    driver.switch_to.window(driver.window_handles[1])
                    
                    # On récupère ici la fonction qui nous permet d'obtenir les détails
                    details = scrape_company_details(driver, wait)
                    if details:
                        record = {
                            "name": name,
                            "grade": details["grade"],
                            "country": details["country"],
                            "date": details["date"],
                            "industry": details["industry"],
                            "employees": details["employees"]
                        }
                        data.append(record)
                    
                        save_data([record], output_file)

                    # Fermer la fenêtre actuelle et revenir à la fenêtre principale
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    
                except Exception as e:
                    print(f"Erreur entreprise {name}: {e}")
                    # Si une erreur se produit, fermer la fenêtre et revenir à la fenêtre principale
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    continue
            
            try: # La suite est le code permettant de changer de page une fois les entreprises
                # de la page actuelle ont été sscreapées

                # On identifie ici le lien pour changer de page
                next_page = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, f"a.pagination-page[href='hrefcurrentpage={page+2}']"))
                )
                # On oublie pas mettre le "+2" juste au-dessus, puisqu'on commence avec page = 0 sur la page 1
                # alors pour arriver à la page suivane qui est la 2ème, on doit ajouter 2

                # La commande suivant simule le clique sur le lien de la page suivante
                driver.execute_script("arguments[0].click();", next_page)
                time.sleep(3)
            except Exception as e:
                print(f"Pas de page suivante ou erreur: {e}")
                break
            except StaleElementReferenceException:
                print("Élément obsolète. Nouvelle tentative...")
                continue


    finally:
        driver.quit() # Une fois le scraping terminé, on ferme le navigateur
    return pd.DataFrame(data)

df = scrape_companies(url,  1391)
print(df)


In [1]:
######################################### BROUILLON DU CODE AVEC FILTRE DE RECHERCHE ########################################################
# driver = webdriver.Chrome()

# # Ouverture de l'URL
# driver.get(url)

# def web_scraping(pages, sector):


#     # Sélection du secteur
#     select_element = Select(driver.find_element(By.ID, "industry")) 
#     select_element.select_by_visible_text(sector)

#     time.sleep(2)

#     # Cliquer sur le bouton Filtrer afin d'appliquer le filtre
#     filter_button = driver.find_element(By.ID, "filterButton")
#     # Simuler le clique
#     filter_button.click()

#     time.sleep(2)

#     # Création des listes recevant les noms et notes ESG des entreprises
#     grades = []
#     names = []

#     # Boucle qui va scraper sur les différentes pages
#     for i in range(pages):
#         grade_elements = driver.find_elements(By.CLASS_NAME, "col-2")
#         name_elements = driver.find_elements(By.CSS_SELECTOR, "a.primary-color.d-block.js-fix-path")

#         grades.append([grade.text for grade in grade_elements])
#         names.append([name.text for name in name_elements])
        
#         # On met un "try" pour que le code fonctionne même si la page suivante n'existe pas
#         try:
            
#             next_page = WebDriverWait(driver, 5).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, f"a.pagination-page[href='hrefcurrentpage={i+2}']"))
#             )
#             driver.execute_script("arguments[0].click();", next_page)
#             time.sleep(3)
#         except Exception as e:
#             print(f"Pas de page suivante ou erreur: {e}")
#             break
#         except StaleElementReferenceException:
#             print("Élément obsolète. Nouvelle tentative...")
#             continue



#     data = {}
#     for x in range(len(names)):
#         for y in range(len(names[x])):
#             if names[x][y] not in data:
#                 data[names[x][y]] = []
#             data[names[x][y]].append(grades[x][y])

#     # Convertir ce dictionnaire en un DataFrame
#     df = pd.DataFrame.from_dict(data, orient='index')
#     df.rename(columns={df.columns[0]: 'Note ESG'}, inplace=True)
#     df.to_csv(f'data/{sector}.csv', index=True, sep='\t')  # Le paramètre 'index=True' permet de sauvegarder les indices (noms des entreprises)
#     print(f'Le DataFrame {sector} a bien été enregistré')

# # sector_list = [
# #     "Aerospace & Defense",
# #     "Auto Components",
# #     "Automobiles",
# #     "Banks",
# #     "Building Products",
# #     "Chemicals",
# #     "Commercial Services",
# #     "Construction & Engineering",
# #     "Construction Materials",
# #     "Consumer Durables",
# #     "Consumer Services",
# #     "Containers & Packaging"
# # ]
# sector_list = [
#     "Diversified Financials",
#     "Diversified Metals",
#     "Electrical Equipment",
#     "Energy Services",
#     "Food Products",
#     "Food Retailers",
#     "Healthcare",
#     "Homebuilders",
#     "Household Products",
#     "Industrial Conglomerates",
#     "Insurance",
#     "Machinery",
#     "Media",
#     "Oil & Gas Producers",
#     "Paper & Forestry",
#     "Pharmaceuticals",
#     "Precious Metals",
#     "Real Estate",
#     "Refiners & Pipelines",
#     "Retailing",
#     "Semiconductors",
#     "Software & Services",
#     "Steel",
#     "Technology Hardware",
#     "Telecommunication Services",
#     "Textiles & Apparel",
#     "Traders & Distributors",
#     "Transportation",
#     "Transportation Infrastructure",
#     "Utilities"
# ]


# for i in range(len(sector_list)):
#     web_scraping(1000, sector_list[i])

# driver.quit()